In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import cv2
import re
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 牛刀小試3



In [39]:
def test_unet(input_size=(256,256,3)):
    inputs = Input(input_size)

    conv1 = Conv2D(32, (3, 3), padding='?', activation='relu', name= 'c1')(inputs)   #256x256x32
    pool1 = MaxPooling2D(pool_size=(?, ?), name= 'c1_M')(conv1)                      #128x128x32

    conv2 = Conv2D(64, (3, 3), padding='?', activation='relu', name= 'c2')(?)        #128x128x32

    up3 = Conv2DTranspose(32, (?, ?), strides=(2, 2), padding='?', name= 'ct3')(conv2)#256x256x32
    up3 = concatenate([up3, ?], axis=3, name= 'c1_concat_ct3')                        #256x256x(32+32)

    conv4 = Conv2D(32, (3, 3), padding='?', activation='relu', name= 'c4')(up3)       #256x256x32
    
    conv5 = Conv2D(?, (?, ?), activation='sigmoid', name= 'c5_sigmoid')(conv4)        #256x256x1

    return Model(inputs=[inputs], outputs=[conv5], name='test_unet')

SyntaxError: invalid syntax (2515454200.py, line 5)

In [2]:
def test_unet(input_size=(256,256,3)):
    inputs = Input(input_size)

    conv1 = Conv2D(32, (3, 3), padding='same', activation='relu', name= 'c1')(inputs)#256x256x32
    pool1 = MaxPooling2D(pool_size=(2, 2), name= 'c1_M')(conv1)                      #128x128x32

    conv2 = Conv2D(64, (3, 3), padding='same', activation='relu', name= 'c2')(pool1) #128x128x32

    up3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', name= 'ct3')(conv2) #256x256x32
    up3 = concatenate([up3, conv1], axis=3, name= 'c1_concat_ct3')                        #256x256x(32+32)

    conv4 = Conv2D(32, (3, 3), padding='same', activation='relu', name= 'c4')(up3)        #256x256x32
    
    conv5 = Conv2D(1, (1, 1), activation='sigmoid', name= 'c5_sigmoid')(conv4)              #256x256x1

    return Model(inputs=[inputs], outputs=[conv5], name='test_unet')

In [4]:
model = test_unet(input_size=(256, 256, 3))
print('Use model:',model.name)
print()
model.summary()

Use model: test_unet

Model: "test_unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
c1 (Conv2D)                     (None, 256, 256, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
c1_M (MaxPooling2D)             (None, 128, 128, 32) 0           c1[0][0]                         
__________________________________________________________________________________________________
c2 (Conv2D)                     (None, 128, 128, 64) 18496       c1_M[0][0]                       
____________________________________________________________________

# 牛刀小試4

## 若是每個pixel分10類?

In [10]:
def test_unet(input_size=(256,256,3)):
    inputs = Input(input_size)

    conv1 = Conv2D(32, (3, 3), padding='?', activation='relu', name= 'c1')(inputs)   #256x256x32
    pool1 = MaxPooling2D(pool_size=(?, ?), name= 'c1_M')(conv1)                      #128x128x32

    conv2 = Conv2D(64, (3, 3), padding='?', activation='relu', name= 'c2')(?)        #128x128x32

    up3 = Conv2DTranspose(32, (?, ?), strides=(2, 2), padding='?', name= 'ct3')(conv2)#256x256x32
    up3 = concatenate([up3, ?], axis=3, name= 'c1_concat_ct3')                        #256x256x(32+32)

    conv4 = Conv2D(32, (3, 3), padding='?', activation='relu', name= 'c4')(up3)       #256x256x32
    
    conv5 = Conv2D(?, (?, ?), activation='softmax', name= 'c5_10_classes')(conv4)           

    return Model(inputs=[inputs], outputs=[conv5], name='test_unet')

SyntaxError: invalid syntax (1982611950.py, line 5)

In [5]:
def test_unet(input_size=(256,256,3)):
    inputs = Input(input_size)

    conv1 = Conv2D(32, (3, 3), padding='same', activation='relu', name= 'c1')(inputs)#256x256x32
    pool1 = MaxPooling2D(pool_size=(2, 2), name= 'c1_M')(conv1)                      #128x128x32

    conv2 = Conv2D(64, (3, 3), padding='same', activation='relu', name= 'c2')(pool1) #128x128x32

    up3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', name= 'ct3')(conv2) #256x256x32
    up3 = concatenate([up3, conv1], axis=3, name= 'c1_concat_ct3')                        #256x256x(32+32)

    conv4 = Conv2D(32, (3, 3), padding='same', activation='relu', name= 'c4')(up3)        #256x256x32
    
    conv5 = Conv2D(10, (1, 1), activation='softmax', name= 'c5_10_classes')(conv4)              #256x256x1

    return Model(inputs=[inputs], outputs=[conv5], name='test_unet')

In [6]:
model = test_unet(input_size=(256, 256, 3))
print('Use model:',model.name)
print()
model.summary()

Use model: test_unet

Model: "test_unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
c1 (Conv2D)                     (None, 256, 256, 32) 896         input_3[0][0]                    
__________________________________________________________________________________________________
c1_M (MaxPooling2D)             (None, 128, 128, 32) 0           c1[0][0]                         
__________________________________________________________________________________________________
c2 (Conv2D)                     (None, 128, 128, 64) 18496       c1_M[0][0]                       
____________________________________________________________________

# Today's architecture

In [7]:
def unet1(input_size=(256,256,3)):
    inputs = Input(input_size)

    conv1 = Conv2D(32, (3, 3), padding='same', name= 'c1_1')(inputs)
    conv1 = BatchNormalization(axis=3, name= 'c1_1_B')(conv1)
    conv1 = Activation('relu', name= 'c1_1_r')(conv1)

    conv1 = Conv2D(32, (3, 3), padding='same', name= 'c1_2')(conv1)
    conv1 = BatchNormalization(axis=3, name= 'c1_2_B')(conv1)
    conv1 = Activation('relu', name= 'c1_2_r')(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2), name= 'c1_2_M')(conv1)

    conv2 = Conv2D(64, (3, 3), padding='same', name= 'c2_1')(pool1)
    conv2 = BatchNormalization(axis=3, name= 'c2_1_B')(conv2)
    conv2 = Activation('relu', name= 'c2_1_r')(conv2)

    conv2 = Conv2D(64, (3, 3), padding='same', name= 'c2_2')(conv2)
    conv2 = BatchNormalization(axis=3, name= 'c2_2_B')(conv2)
    conv2 = Activation('relu', name= 'c2_2_r')(conv2)

    pool2 = MaxPooling2D(pool_size=(2, 2), name= 'c2_2_M')(conv2)

    conv3 = Conv2D(128, (3, 3), padding='same', name= 'c3_1')(pool2)
    conv3 = BatchNormalization(axis=3, name= 'c3_1_B')(conv3)
    conv3 = Activation('relu', name= 'c3_1_r')(conv3)

    conv3 = Conv2D(128, (3, 3), padding='same', name= 'c3_2')(conv3)
    conv3 = BatchNormalization(axis=3, name= 'c3_2_B')(conv3)
    conv3 = Activation('relu', name= 'c3_2_r')(conv3)

    pool3 = MaxPooling2D(pool_size=(2, 2), name= 'c3_2_M')(conv3)

    conv4 = Conv2D(256, (3, 3), padding='same', name= 'c4_1')(pool3)
    conv4 = BatchNormalization(axis=3, name= 'c4_1_B')(conv4)
    conv4 = Activation('relu', name= 'c4_1_r')(conv4)

    conv4 = Conv2D(256, (3, 3), padding='same', name= 'c4_2')(conv4)
    conv4 = BatchNormalization(axis=3, name= 'c4_2_B')(conv4)
    conv4 = Activation('relu', name= 'c4_2_r')(conv4)

    pool4 = MaxPooling2D(pool_size=(2, 2), name= 'c4_2_M')(conv4)

    conv5 = Conv2D(512, (3, 3), padding='same', name= 'c5_1')(pool4)
    conv5 = BatchNormalization(axis=3, name= 'c5_1_B')(conv5)
    conv5 = Activation('relu', name= 'c5_1_r')(conv5)

    conv5 = Conv2D(512, (3, 3), padding='same', name= 'c5_2')(conv5)
    conv5 = BatchNormalization(axis=3, name= 'c5_2_B')(conv5)
    conv5 = Activation('relu', name= 'c5_2_r')(conv5)

    up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', name= 'ct6')(conv5)
    up6 = concatenate([up6, conv4], axis=3, name= 'ct6_concat_c4_2_r')

    conv6 = Conv2D(256, (3, 3), padding='same', name= 'c6_1')(up6)
    conv6 = BatchNormalization(axis=3, name= 'c6_1_B')(conv6)
    conv6 = Activation('relu', name= 'c6_1_r')(conv6)

    conv6 = Conv2D(256, (3, 3), padding='same', name= 'c6_2')(conv6)
    conv6 = BatchNormalization(axis=3, name= 'c6_2_B')(conv6)
    conv6 = Activation('relu', name= 'c6_2_r')(conv6)

    up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name= 'ct7')(conv6)
    up7 = concatenate([up7, conv3], axis=3, name= 'ct7_concat_c3_2_r')

    conv7 = Conv2D(128, (3, 3), padding='same', name= 'c7_1')(up7)
    conv7 = BatchNormalization(axis=3, name= 'c7_1_B')(conv7)
    conv7 = Activation('relu', name= 'c7_1_r')(conv7)

    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', name= 'c7_2')(conv7)
    conv7 = BatchNormalization(axis=3, name= 'c7_2_B')(conv7)
    conv7 = Activation('relu', name= 'c7_2_r')(conv7)

    up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', name= 'ct8')(conv7)
    up8 = concatenate([up8, conv2], axis=3, name= 'ct8_concat_c2_2_r')

    conv8 = Conv2D(64, (3, 3), padding='same', name= 'c8_1')(up8)
    conv8 = BatchNormalization(axis=3, name= 'c8_1_B')(conv8)
    conv8 = Activation('relu', name= 'c8_1_r')(conv8)

    conv8 = Conv2D(64, (3, 3), padding='same', name= 'c8_2')(conv8)
    conv8 = BatchNormalization(axis=3, name= 'c8_2_B')(conv8)
    conv8 = Activation('relu', name= 'c8_2_r')(conv8)

    up9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', name= 'ct9')(conv8)
    up9 = concatenate([up9, conv1], axis=3, name= 'ct9_concat_c1_2_r')

    conv9 = Conv2D(32, (3, 3), padding='same', name= 'c9_1')(up9)
    conv9 = BatchNormalization(axis=3, name= 'c9_1_B')(conv9)
    conv9 = Activation('relu', name= 'c9_1_r')(conv9)

    conv9 = Conv2D(32, (3, 3), padding='same', name= 'c9_2')(conv9)
    conv9 = BatchNormalization(axis=3, name= 'c9_2_B')(conv9)
    conv9 = Activation('relu', name= 'c9_2_r')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid', name= 'c10_sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10], name='unet1')

In [8]:
model = unet1(input_size=(256, 256, 3))
# model = unet2(input_size=(im_height, im_width, 3))
print('Use model:',model.name)
print()
model.summary()

Use model: unet1

Model: "unet1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
c1_1 (Conv2D)                   (None, 256, 256, 32) 896         input_4[0][0]                    
__________________________________________________________________________________________________
c1_1_B (BatchNormalization)     (None, 256, 256, 32) 128         c1_1[0][0]                       
__________________________________________________________________________________________________
c1_1_r (Activation)             (None, 256, 256, 32) 0           c1_1_B[0][0]                     
____________________________________________________________________________

## Unet1 Architecture
<br>
<img src="unet1.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" width="1000"/>